In [354]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone

In [355]:
df_churn_or = pd.read_csv('./provjera/provjera_churn_or.csv', sep=',', header=None, names=['player_id', 'churn'])

In [356]:
# Read data
df_zadatak = pd.read_csv('./provjera/provjera_zadatak.csv', sep=';', header=None, names=['player_id', 'trans_hour', 'product', 'transaction_type', 'amount', 'cnt'])

In [357]:
df_igraci = pd.read_csv('./provjera/provjera_igraci.csv', sep=',', header=None, names=['player_id', 'birth_date', 'city', 'registration_date', 'registration_hour', 'is_opt_out', 'registration_terminal'])

In [358]:
df_zadatak['trans_hour'] = pd.to_datetime(df_zadatak['trans_hour']).dt.tz_localize(None)

In [359]:
# def player_total_payout_per_product(df_zadatak, current_date: pd.Timestamp):
#     """Returns a dataset of product total payout"""
#     df_zadatak
#     df_zadatak = df_zadatak[df_zadatak['transaction_type'] == 'TicketWin']
#     df_zadatak = df_zadatak.groupby(['player_id', 'product']).sum()
#     return df_zadatak

# def player_total_payin_per_product(df_zadatak, current_date):
#     """Returns a dataset of product total payin"""
#     df_zadatak = df_zadatak[df_zadatak['transaction_type'] == 'TicketPayin']
#     df_zadatak = df_zadatak.groupby(['player_id', 'product']).sum()
#     return df_zadatak

In [360]:
def all_transactions_in_last_30_days(df, current_date):
    """Returns a dataset of all transactions in last 30 days"""
    df = df[df['trans_hour'].between(current_date - pd.Timedelta(days=40), current_date)]
    return df

In [361]:
def total_win_for_player(df, current_date):
    """Returns a dataset of total profit for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'TicketWin']
    df = df.groupby(['player_id']).sum()
    df.rename(columns={'amount': 'total_win'}, inplace=True)
    df.rename(columns={'cnt': 'cnt_win'}, inplace=True)
    return df

def total_payin_for_player(df, current_date):
    """Returns a dataset of total payin for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'TicketPayin']
    df = df.groupby(['player_id']).sum()
    df.rename(columns={'amount': 'total_payin'}, inplace=True)
    df.rename(columns={'cnt': 'cnt_payin'}, inplace=True)
    return df

In [362]:
def total_profit_for_player(df, current_date):
    """Returns a dataset of total profit for each player"""
    df = pd.merge(total_win_for_player(df, current_date), total_payin_for_player(df, current_date), how='right', on='player_id')

    # fill all NAn values with 0
    df.fillna(0, inplace=True)

    # calculate total profit
    df['total_profit'] = df['total_win'] - df['total_payin']
    return df['total_profit'].to_frame()

In [363]:
def total_deposit_for_player(df, current_date):
    """Returns a dataset of total deposit for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'Deposit']
    df = df.groupby(['player_id']).sum()
    df.rename(columns={'amount': 'total_deposit'}, inplace=True)
    df.rename(columns={'cnt': 'cnt_deposit'}, inplace=True)

    #fill all NAn values with 0
    df.fillna(0, inplace=True)
    return df

def total_withdrawal_for_player(df, current_date):
    """Returns a dataset of total withdrawal for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'Withdrawal']
    df = df.groupby(['player_id']).sum()
    df.rename(columns={'amount': 'total_withdrawal'}, inplace=True)
    df.rename(columns={'cnt': 'cnt_withdrawal'}, inplace=True)

    #fill all NAn values with 0
    df.fillna(0, inplace=True)
    return df

def total_bonus_for_player(df, current_date):
    """Returns a dataset of total bonus for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'Bonus']
    df = df.groupby(['player_id']).sum()
    df.rename(columns={'amount': 'total_bonus'}, inplace=True)
    df.rename(columns={'cnt': 'cnt_bonus'}, inplace=True)

    #fill all NAn values with 0
    df.fillna(0, inplace=True)
    
    return df

def total_deposit_minus_withdrawal_for_player(df, current_date):
    """Returns a dataset of total account for each player"""
    df = pd.merge(total_deposit_for_player(df, current_date), total_withdrawal_for_player(df, current_date), how='left', on='player_id')
    
    # fill all NAn values with 0
    df.fillna(0, inplace=True)
    return df

In [364]:
# total_deposit_minus_withdrawal_for_player(df_zadatak, current_date)

def total_account_without_profit_for_player(df, current_date):
    """Returns a dataset of total account for each player"""
    df = pd.merge(total_deposit_minus_withdrawal_for_player(df, current_date), total_bonus_for_player(df, current_date), how='left', on='player_id')
    
    # fill all NAn values with 0
    df.fillna(0, inplace=True)

    # calculate total account
    df['total_account'] = df['total_deposit'] - df['total_withdrawal'] + df['total_bonus']
    return df['total_account'].to_frame()

In [365]:
def total_account_for_player(df, current_date):
    """Returns a dataset of total account for each player"""
    df = pd.merge(total_account_without_profit_for_player(df, current_date), total_profit_for_player(df, current_date), how='right', on='player_id')
    
    # fill all NAn values with 0
    df.fillna(0, inplace=True)

    # calculate total account
    df['total_account'] = df['total_account'] + df['total_profit']

    # round total account to 2 decimal places
    df['total_account'] = df['total_account'].round(2)
    return df['total_account'].to_frame()

In [366]:
def total_days_played_for_player_in_last_30(df, current_date):
    """Returns a dataset of total days played for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'TicketPayin']
    df['trans_hour'] = df['trans_hour'].dt.date
    df = df.groupby(['player_id', 'trans_hour']).count()
    df = df.groupby(['player_id']).count()
    df.rename(columns={'product': 'total_days_played'}, inplace=True)

    #fill all NAn values with 0
    df.fillna(0, inplace=True)
    
    return df

# total_days_played_for_player_in_last_30(df_zadatak, current_date).head(50)

In [367]:
def days_since_last_win_for_player(df, current_date):
    """Returns a dataset of days since last win for each player"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'TicketWin']
    df = df.groupby(['player_id'])
    df = df['trans_hour'].max()
    df = df.to_frame()
    df['trans_hour'] = df['trans_hour'].apply(lambda x: current_date - x)
    df['trans_hour'] = df['trans_hour'].apply(lambda x: x.days)
    # rename trans hour to days since last win
    df.rename(columns={'trans_hour': 'days_since_last_win'}, inplace=True)

    #fill all NAn values with 31
    df.fillna(0, inplace=True)
    return df


In [368]:
def win_loss_ratio(df, current_date):
    """Returns a dataset of win loss ratio for each player"""
    df = pd.merge(total_win_for_player(df, current_date), total_payin_for_player(df, current_date), how='right', on='player_id')
    
    # fill all NAn values with 0
    df.fillna(0, inplace=True)

    # calculate win loss ratio
    df['win_loss_ratio'] = df['cnt_win'] / df['cnt_payin']
    return df

In [369]:
def over_five_deposits(df, current_date):
    """Returns a dataset of players who did deposit in last 30 days"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'Deposit']
    df = df.groupby(['player_id']).count()
    df['over_five_deposits'] = df['cnt'] > 5
    df = df['over_five_deposits'].to_frame()
    df['over_five_deposits'] = df['over_five_deposits'].apply(lambda x: 1 if x else 0)
    return df

In [370]:
def over_thirty_years_old(df, current_date):
    """Returns a dataset of players who are over 30 years old"""

    df['birth_date'] = pd.to_datetime(df['birth_date'])

    df = df.groupby(['player_id']).max()
    df['age'] = df['birth_date'].apply(lambda x: current_date - x)
    df['age'] = df['age'].apply(lambda x: x.days / 365)
    df['over_thirty_years_old'] = df['age'] > 30
    df = df['over_thirty_years_old'].to_frame()
    df['over_thirty_years_old'] = df['over_thirty_years_old'].apply(lambda x: 1 if x else 0)
    return df

In [371]:
def registered_for_over_a_year(df, current_date):
    """Returns a dataset of players who registered for over a year"""
    df = df.groupby(['player_id']).max()
    df['registration_date'] = pd.to_datetime(df['registration_date'])
    df['registered_for_over_a_year'] = df['registration_date'].apply(lambda x: current_date - x)
    df['registered_for_over_a_year'] = df['registered_for_over_a_year'].apply(lambda x: x.days / 365)
    df['registered_for_over_a_year'] = df['registered_for_over_a_year'] > 1
    df = df['registered_for_over_a_year'].to_frame()
    df['registered_for_over_a_year'] = df['registered_for_over_a_year'].apply(lambda x: 1 if x else 0)
    return df

In [372]:
def registered_for_over_a_month(df, current_date):
    """Returns a dataset of players who registered for over a month"""
    df = df.groupby(['player_id']).max()
    df['registration_date'] = pd.to_datetime(df['registration_date'])
    df['registered_for_over_a_month'] = df['registration_date'].apply(lambda x: current_date - x)
    df['registered_for_over_a_month'] = df['registered_for_over_a_month'].apply(lambda x: x.days / 30)
    df['registered_for_over_a_month'] = df['registered_for_over_a_month'] > 1
    df = df['registered_for_over_a_month'].to_frame()
    df['registered_for_over_a_month'] = df['registered_for_over_a_month'].apply(lambda x: 1 if x else 0)
    return df

In [373]:
# how many different types of games did player play in last 30 days
def different_games_played_in_last_30_days(df, current_date):
    """Returns a dataset of how many different types of games did player play in last 30 days"""
    df = all_transactions_in_last_30_days(df, current_date)
    df = df[df['transaction_type'] == 'TicketPayin']
    df = df.groupby(['player_id', 'product']).count()
    df = df.groupby(['player_id']).count()
    df.rename(columns={'cnt': 'different_games_played_in_last_30_days'}, inplace=True)
    return df


In [374]:
def reg_on_mobile_web(df, current_date):
    """Returns a dataset of players who registered on mobile web"""
    df = df.groupby(['player_id']).max()
    df['registration_terminal'] = df['registration_terminal'].apply(lambda x: 1 if x == ('MobileWeb' or 'Mobile Web') else 0)
    df.rename(columns={'registration_terminal': 'reg_on_mobile_web'}, inplace=True)
    df.drop(['is_opt_out', 'registration_hour', 'registration_date', 'city', 'birth_date'], axis=1, inplace=True)
    return df

In [375]:
current_date = df_zadatak['trans_hour'].max().date().strftime('%d/%m/%Y')
current_date = pd.to_datetime(current_date, dayfirst=True).to_datetime64()
current_date

numpy.datetime64('2022-11-30T00:00:00.000000000')

In [376]:
def dfForAppend(df_zadatak, df_igraci, current_date):
    df = df_igraci
    df = pd.merge(df, total_account_for_player(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, total_days_played_for_player_in_last_30(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, days_since_last_win_for_player(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, total_profit_for_player(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, win_loss_ratio(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, over_five_deposits(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, different_games_played_in_last_30_days(df_zadatak, current_date), how='left', on='player_id')
    df = pd.merge(df, reg_on_mobile_web(df_igraci, current_date), how='left', on='player_id')
    
    # drop coloumns city registration_hour registration terminal product and count
    df.drop(['registration_hour', 'city', 'cnt', 'total_win', 'cnt_win', 'total_payin', 'cnt_payin', 'transaction_type_x', 'amount_x', 'transaction_type_y', 'amount_y', 'trans_hour', 'registration_terminal'], axis=1, inplace=True)

    # change birth date to age
    df['age'] = df['birth_date'].apply(lambda x: current_date - pd.to_datetime(x))
    df['age'] = df['age'].apply(lambda x: int((x.days / 365)))

    # change registration date to yyyy-mm-dd
    df['registration_date'] = pd.to_datetime(df['registration_date']).dt.year

    # substract 2000 from registration date
    df['registration_date'] = df['registration_date'].apply(lambda x: x - 2014)

    df.drop(['birth_date'], axis=1, inplace=True)
    
    return df

In [377]:
df = dfForAppend(df_zadatak, df_igraci, current_date)

/var/folders/3p/d4b3b1sx3db26ptpggsb08w80000gn/T/ipykernel_24209/4078790061.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['player_id']).sum()
/var/folders/3p/d4b3b1sx3db26ptpggsb08w80000gn/T/ipykernel_24209/4078790061.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['player_id']).sum()
/var/folders/3p/d4b3b1sx3db26ptpggsb08w80000gn/T/ipykernel_24209/4078790061.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which 

In [378]:
df['days_since_last_win'] = df['days_since_last_win'].fillna(31)
df['over_five_deposits'] = df['over_five_deposits'].fillna(0)

df = df[df['win_loss_ratio'] <= 1]

df.to_csv('./export/test_data_processed.csv', index=False)

In [379]:
df['player_id'].value_counts()

2       1
1896    1
1848    1
1860    1
1865    1
       ..
777     1
783     1
784     1
785     1
3237    1
Name: player_id, Length: 1002, dtype: int64

In [380]:
df = pd.merge(df, df_churn_or, how='left', on='player_id')

In [381]:
df.dropna(inplace=True)

In [382]:
df

,player_id,registration_date,is_opt_out,total_account,total_days_played,days_since_last_win,total_profit,win_loss_ratio,over_five_deposits,different_games_played_in_last_30_days,reg_on_mobile_web,age,churn
0,2,0,True,-3.80,13.0,5.0,-48.95,0.270833,1.0,3.0,1,30,0.0
1,3,0,False,-2.00,4.0,7.0,31.49,0.278481,0.0,2.0,0,51,0.0
2,4,0,False,-29.88,26.0,11.0,-52.48,0.046512,0.0,1.0,0,49,0.0
3,5,0,False,-0.20,13.0,1.0,-45.42,0.869344,1.0,3.0,0,30,0.0
4,6,0,False,-0.48,3.0,14.0,-160.19,0.719465,1.0,1.0,0,51,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1045,6,False,0.41,4.0,23.0,-7.41,0.234043,0.0,1.0,0,48,0.0
416,1050,6,False,-0.80,5.0,15.0,-13.23,0.821839,0.0,3.0,0,44,1.0
417,1051,6,False,-1.04,5.0,6.0,-8.95,0.829730,0.0,4.0,0,20,1.0
418,1054,6,False,-3.70,29.0,0.0,107.50,0.738623,1.0,3.0,0,32,0.0


In [383]:
df.to_csv('./export/test_data_processed_with_churn.csv', index=False)